In [4]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "cameron12"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date


app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Center(html.B(html.H1('Cameron Holloman SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(className = 'row',
        style = {'display' : 'flex'},
        children=[
                html.Button(id='submit-button-one', n_clicks=0, children = 'Water Rescue'),
                html.Button(id='submit-button-two', n_clicks=0, children = 'Mountain/Wilderness Rescue'),
                html.Button(id='submit-button-three', n_clicks=0, children = 'Disaster or Individual Tracking'),
                html.Button(id='submit-button-four', n_clicks=0, children = 'Reset')
        ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size= 10
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('submit-button-one', 'n_clicks'),
               Input('submit-button-two', 'n_clicks'),
               Input('submit-button-three', 'n_clicks'),
               Input('submit-button-four', 'n_clicks'),
              ])
def update_dashboard(bt1,bt2,bt3,bt4):
### FIX ME Add code to filter interactive data table with MongoDB queries
    if (int(bt1 == 0) and int(bt2 == 0) and int(bt3 == 0) and int(bt4 == 0)):
        df = pd.DataFrame.from_records(shelter.read({}))
        
    elif(int(bt1) >= 1):
        df = pd.DataFrame.from_records(list(shelter.read({
            "animal_type":"Dog",
            "breed":{"$in":["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcomes": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte":26},
            "age_upon_outcome_in_weeks": {"$lte":156}
        })))
        bt2, bt3, bt4 = 0

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    
    
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame = dff,
                values = ['52.2', '17.6', '6.79', '5.8', '5.8', '5.78', '2.87', '2', '1.19'],
                names = ['Miniature Schnauzer Mix', 'Chihuahua Shorthair Mix', 'Pitbull', 'Domestic Medium Hair Mix', 'Beagle Mix', 'Squirrel Mix', 'Bat Mix', 'Leonberger Mix', 'Domestic Shorthair Mix'],
                color_discrete_sequence = px.colors.sequential.RdBu,
                width = 800,
                height = 500
            )
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1('animal_name'),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]


app